In [81]:
def best_paramssss(train_features, train_labels):
    
    ######################    
    
    ######################
    
    return best_param

In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from math import *
from scipy.stats import mode

from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

ImportError: cannot import name 'QuantileRegressor' from 'sklearn.linear_model' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\__init__.py)

In [2]:
zs= 0.53

def accuracy_calculate(actual_values, predicted_values):
    comparison = abs(np.round(predicted_values) - actual_values)
    accuracy = 1- ((len(comparison[comparison>=(0+1)])) / len(actual_values))
    return accuracy

def mean_iou_calculator(actual_values, predicted_values, time):
    confusion_array = confusion_matrix(actual_values, predicted_values)
    individual_ious = []
    for i in range(len(confusion_array)):
        individual_iou = confusion_array[i][i] / (sum(confusion_array[i]))
        individual_ious.append(individual_iou)
    mean_iou = sum(individual_ious)/len(individual_ious)
    results = pd.DataFrame()  
    featue_y_values = sorted(data[time].unique())
    for i in range(len(individual_ious)):
        results.insert(0, 'iou_(' + str(featue_y_values[i])  +')', [individual_ious[i]], True)
    results = results[results.columns[::-1]]
    results.insert(0, 'mean_iou', mean_iou, True)
    return results

data = pd.read_csv('gulle.csv', sep=';', encoding = "utf8")
data.columns = ['product', 'amount', 'company', 'town', 'type', 'order date', 'delivery date', 'time'] #Rearannge the dataframe as the old one

data2 = data.drop('delivery date', axis = 1)
data2["order day"] = ''
data2["order month"] = ''
data2["week day"] = ''


# Remove 'order date' and add 'order day', 'order month' and 'week day' features
for i in range(len(data2)):
  data2.at[i, 'order day'] = data2['order date'][i].split()[0]
  data2.at[i, 'order month'] = data2['order date'][i].split()[1]
  data2.at[i, 'week day'] = data2['order date'][i].split()[-1]
data2 = data2.drop('order date', axis = 1)
data2['week day'] = data2['week day'].str.replace('Pazartesi','pts')
data2['week day'] = data2['week day'].str.replace('Salı','sal')
data2['week day'] = data2['week day'].str.replace('Çarşamba','çrş')
data2['week day'] = data2['week day'].str.replace('Perşembe','prş')
data2['week day'] = data2['week day'].str.replace('Cumartesi','cts')
data2['week day'] = data2['week day'].str.replace('Cuma','cum')
data2['week day'] = data2['week day'].str.replace('Pazar','paz')

# data2 = data2[data2["week day"].str.contains("Pazar") == False]

# Rearranging Dataframe
data2 = data2[['product', 'company', 'amount', 'town', 'type', 'order day', 'week day', 'order month', 'time']]
data2['town'] = data2['town'].str.lower()
data2['town'] = data2['town'].str.replace('i̇','i')
data2['town'] = data2['town'].str.replace('.','missing')
data2['town'] = data2['town'].str.replace(' tekirdağ','tekirdağ')
data2['town'] = data2['town'].str.replace('küçükçekmece','istanbul')
data2['town'] = data2['town'].str.replace('çorlu','tekirdağ')
data2['town'] = data2['town'].str.replace('bandirma','balıkesir')

#data2 = data2.drop('town',axis = 1).reset_index(drop=True)
data2 = data2.drop('order day',axis = 1).reset_index(drop=True)
data2 = data2.drop('order month',axis = 1).reset_index(drop=True)
data2 = data2.fillna("missing")
#data2 = data2[data2["town"].str.contains("missing") == False]

data_clean = data2.copy()
drop_df = data2.copy()
drop_index_list = []

def remove_insuff(df, ft):
    print("removing insufficient data for ", ft, "...")
    fst_len = len(df)
    x = df[ft].value_counts() < 5 
    df2 = x.to_frame().reset_index()
    df2.columns = [ft, 'booly']
    df2.drop(df2[df2.booly == False].index, inplace=True)
    drop_list = df2[ft].tolist()
    drop_indices=[]

    if len(drop_list) != 0:
        for i in df.index:
            for j in range(len(drop_list)):
                if (drop_list[j] == df.at[i, ft]):
                    drop_indices = drop_indices + [i]
        df.drop(drop_indices, inplace=True)
        
    else:
        drop_indices = []
                        
    lst_len = len(df)
    rem = fst_len - lst_len      # number of removed data
    per = (rem / fst_len) * 100  # percentage of removed data

    print("total number of removed data: ", rem)
    print("persentage of removed data: ", round(per, 2), "%")
    return df, drop_indices

data2, drop_indices = remove_insuff(data2, "company")
drop_index_list = drop_index_list + drop_indices

data2, drop_indices = remove_insuff(data2, "product")
drop_index_list = drop_index_list + drop_indices

data_suf = data2.copy()

n_prod = data_suf["product"].nunique()
prod_list = data_suf["product"].unique()
n_comp = data_suf["company"].nunique()
comp_list = data_suf["company"].unique()

def one_hot(df, ft):      ### ft = "company", "product", "week day" etc.
    print("one hot encoding ", ft, "...")
    dum = pd.get_dummies(df[ft])
    df = df.drop(ft, axis = 1)
    df = df.join(dum)
    print(ft, "encoded.")
    return df

data2  = one_hot(data2, "week day")
data2  = one_hot(data2, "company")
data2  = one_hot(data2, "product")
data2

from sklearn.model_selection import train_test_split

df4 = data2
x_train, x_val = train_test_split(df4, test_size = 0.2, random_state = 40)

#train_features, test_features = x_train, x_val

# train_labels, test_labels = y_train, y_val

def clean_noise(df): # df = x_train/x_test
    in_len = len(df)
        
    print("Cleaning noise ... ")
    
    index_drop_list = []
    for prod in prod_list:

        df_max_scaled = df[df[prod] == 1].copy()

        for comp in comp_list:
            df_max_scaled2 = df_max_scaled[df_max_scaled[comp] == 1].copy()

            if len(df_max_scaled2) > 1:
                
                max_min_t = df_max_scaled2["time"].max() - df_max_scaled2["time"].min()
                max_min_a = df_max_scaled2["amount"].max() - df_max_scaled2["amount"].min()
                
                if (max_min_a != 0) and (max_min_t != 0):
                    df_max_scaled2["time"] = (df_max_scaled2["time"] - df_max_scaled2["time"].min()) / max_min_t
                    t_sc = df_max_scaled2[["time"]]
                    df_zscore_t = (t_sc - t_sc.mean())/t_sc.std()
                    dfz_t = abs(df_zscore_t) > zs

                    df_max_scaled2["amount"] = (df_max_scaled2["amount"] - df_max_scaled2["amount"].min()) / max_min_a
                    amo_sc = df_max_scaled2["amount"]
                    df_zscore_a = (amo_sc - amo_sc.mean())/amo_sc.std()
                    dfz_a = abs(df_zscore_a) > zs

                    df1 = dfz_t["time"] & dfz_a 
                    df2 = (df_zscore_t["time"] * df_zscore_a) < 0 
                    dfz = df1 & df2 

                    index_drop_list = index_drop_list + [*filter(dfz.get, dfz.index)]

    index_drop_list = sorted(list(set(index_drop_list)))
    df.drop(index_drop_list, axis=0, inplace=True)
    rem = len(index_drop_list)
    f_len = len(df)
    n_del = in_len - f_len
    per = (n_del / in_len) * 100
    
    # print("deleted indices: ",index_drop_list)
    print("total number of removed data: ", n_del)
    print("persentage of removed data: ", round(per, 2), "%")
    
    return df, index_drop_list

pd.options.mode.chained_assignment = None

x_train, train_drop_list = clean_noise(x_train)
drop_index_list = drop_index_list + train_drop_list

x_val, val_drop_list = clean_noise(x_val)
drop_index_list = drop_index_list + val_drop_list

data2.drop(train_drop_list, axis=0, inplace=True)
data2.drop(val_drop_list, axis=0, inplace=True)

drop_index_list = sorted(list(set(drop_index_list)))
drop_df = drop_df.loc[drop_index_list]

data_clean.drop(drop_df.index.to_list(), axis=0, inplace=True)

x_train = x_train.drop('town',axis = 1).reset_index(drop=True)
x_val = x_val.drop('town',axis = 1).reset_index(drop=True)

# def map_time(df):
#     for i in df.index:
#         if (df.at[i, "time"]==1):
#             df.at[i, "time"] = 7
#         elif (df.at[i, "time"]==2):
#             df.at[i, "time"] = 6
#         elif ((df.at[i, "time"]<=4) and (df.at[i, "time"]>=3)):
#             df.at[i, "time"] = 5
#         elif ((df.at[i, "time"]<=7) and (df.at[i, "time"]>=5)):
#             df.at[i, "time"] = 4
#         elif ((df.at[i, "time"]<=14) and (df.at[i, "time"]>=8)):
#             df.at[i, "time"] = 3
#         elif ((df.at[i, "time"]<=30) and (df.at[i, "time"]>=15)):
#             df.at[i, "time"] = 2
#         elif (df.at[i, "time"]>30):
#             df.at[i, "time"] = 1
            
# map_time(x_train)
# map_time(x_val)
            
#######################################  

# def categorize(df):
#     ls = df.columns.to_list()
#     ls.remove("amount")
#     df[ls] = df[ls].astype('category')
#     #df['time'] = df['time'].cat.rename_categories({1:3, 2:1, 3:4, 4:2, 5:76, 6:32, 7:-6})
#     return df

# x_train = categorize(x_train)
# x_val = categorize(x_val)

################################3
            

y_train = x_train["time"].copy().to_frame()
x_train.drop("time", axis=1, inplace=True)

y_val = x_val["time"].copy().to_frame()
x_val.drop("time", axis=1, inplace=True)

y_train = y_train.squeeze(axis=1)
y_val = y_val.squeeze(axis=1)

y_val = y_val.to_list()

xt_min = x_train["amount"].min()
xt_max = x_train["amount"].max()

x_train["amount"] = (x_train["amount"] - xt_min) / (xt_max - xt_min)
x_val["amount"] = (x_val["amount"] - xt_min) / (xt_max - xt_min)

df_empty = x_train[0:0]

def do_pca(x_train, x_val):
    print("Doing PCA")
    from sklearn.decomposition import PCA
    xl = len(x_train.columns)
    pca = PCA(.95)
    pca.fit(x_train)
    print("number of features dropped from ", xl, " to ", pca.n_components_) 
    #print("variance ratio: ", pca.explained_variance_ratio_) 

    x_train = pca.transform(x_train)
    x_val = pca.transform(x_val)
    return pca, x_train, x_val

def get_categorical_fts(df):
    fts = list(df.columns)
    fts.remove('amount')
    return fts

def do_smotenc(x_train, y_train):
    print('Doing SMOTENC to the training dataset')
    from imblearn.over_sampling import SMOTENC
    from collections import Counter

    print(f'Original dataset shape {x_train.shape}')
    print(f'Original dataset samples per class {Counter(y_train)}')

    # simulate the 2 last columns to be categorical features
    fts = list(range(1, x_train.shape[1]))

    sm = SMOTENC(random_state=42, categorical_features=fts, k_neighbors=4)
    X_res, y_res = sm.fit_resample(x_train, y_train)
    print(f'Resampled dataset samples per class {Counter(y_res)}')
    print(f'Resampled dataset shape {X_res.shape}')
    x_train, y_train = X_res, y_res
    return x_train, y_train

#x_train, y_train = do_smotenc(x_train, y_train) # SMOTENC works only before PCA
pca, x_train, x_val = do_pca(x_train, x_val)
print("\ndone...")


removing insufficient data for  company ...
total number of removed data:  32
persentage of removed data:  2.87 %
removing insufficient data for  product ...
total number of removed data:  169
persentage of removed data:  15.59 %
one hot encoding  week day ...
week day encoded.
one hot encoding  company ...
company encoded.
one hot encoding  product ...
product encoded.
Cleaning noise ... 
total number of removed data:  90
persentage of removed data:  12.3 %
Cleaning noise ... 
total number of removed data:  24
persentage of removed data:  13.11 %
Doing PCA
number of features dropped from  84  to  43

done...


In [3]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pprint import pprint

# Examine the Default Random Forest to Determine Parameters

In [23]:
train_features, test_features = x_train, x_val
train_labels, test_labels = y_train, y_val

In [28]:
rf = Lasso(random_state = 42)
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': 42,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}


# Random Search with Cross Validation

In [9]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]
n_estimators.append(2000)
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 4)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

n_set = len(n_estimators) * len(max_depth) * len(max_features) * len(min_samples_leaf) * len(min_samples_split)
pprint(random_grid)

{'max_depth': [10, 43, 76, 110, None],
 'max_features': ['log2', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 2000]}


In [13]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = 42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = ceil(0.1 * n_set), scoring='neg_mean_absolute_error', 
                              cv = 2, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(train_features, train_labels);

Fitting 2 folds for each of 54 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  1.7min finished


In [14]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'log2',
 'max_depth': 110}

In [15]:
rf_random.best_estimator_

RandomForestRegressor(max_depth=110, max_features='log2', min_samples_leaf=4,
                      min_samples_split=10, n_estimators=400, random_state=42)

# Evaluation Function

In [29]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

### Evaluate the Default Model

In [30]:
base_model = rf
base_model.fit(train_features, train_labels)
base_accuracy = evaluate(base_model, test_features, test_labels)

Model Performance
Average Error: 2.1295 degrees.
Accuracy = -22.90%.


# R2

In [31]:
from sklearn.metrics import r2_score

y_pred = base_model.predict(test_features)
y_true = test_labels
r2_score(y_true, y_pred)



-0.004944081684696622

# Evaluate the Best Random Search Model

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_features, test_labels)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

# Grid Search

In [ ]:
rf_random.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a base model
rf = RandomForestRegressor(random_state = 42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [37]:
#### Aşağıdaki param_grid oluşturulurken yukarıdaki rf_random.best_params_ çıktısını kullanacak şekilde yap!

In [75]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [16, 18, 20],
    'max_features': ['sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [4, 5, 6],
    'n_estimators': [10, 200],
    'class_weight': class_weight
}

# Create a base model
rf = RandomForestClassifier(random_state = 42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, scoring = 'balanced_accuracy', param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [76]:
# Fit the grid search to the data
grid_search.fit(train_features, train_labels);

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   40.5s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:   42.2s finished


In [77]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 18,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 4,
 'n_estimators': 10}

In [78]:
rf_random.best_params_

{'n_estimators': 10,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 18,
 'bootstrap': False}

# Evaluate the Best Model from Grid Search

In [79]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, test_features, test_labels)

Accuracy = 0.58%.
Balanced Accuracy = 0.23%.


In [80]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

Improvement of -5.72%.


# 2nd Round of Grid Search

In [24]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 18,
 'max_features': 'log2',
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'n_estimators': 200}

In [25]:
param_grid = {
    'bootstrap': [False],
    'max_depth': [17, 18, 19],
    'max_features': ['log2'],
    'min_samples_leaf': [6, 7, 8],
    'min_samples_split': [1, 2, 3],
    'n_estimators': [180, 200, 220],
    'class_weight': class_weight
}

# Create a base model
rf = RandomForestClassifier(random_state = 42)

# Instantiate the grid search model
grid_search_final = GridSearchCV(estimator = rf, scoring = 'balanced_accuracy', param_grid = param_grid, 
                                 cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [26]:
grid_search_final.fit(train_features, train_labels);

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:   42.6s finished


In [27]:
grid_search_final.best_params_

{'bootstrap': False,
 'max_depth': 17,
 'max_features': 'log2',
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'n_estimators': 180}

In [28]:
best_grid_final = grid_search_final.best_estimator_
grid_final_accuracy = evaluate(best_grid_final, test_features, test_labels)

Accuracy = 0.67%.
balanced_accuracy:  0.28811928811928805


In [29]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_final_accuracy - grid_accuracy) / grid_accuracy))

Improvement of 2.11%.


# 3rd Round of Grid Search

In [30]:
grid_search_final.best_params_

{'bootstrap': False,
 'max_depth': 17,
 'max_features': 'log2',
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'n_estimators': 180}

In [31]:
param_grid = {
    'bootstrap': [False],
    'max_depth': [15, 16, 17],
    'max_features': ['log2'],
    'min_samples_leaf': [4, 5, 6],
    'min_samples_split': [1, 2, 3],
    'n_estimators': [120, 150, 180],
    'class_weight': class_weight
}

# Create a base model
rf = RandomForestClassifier(random_state = 42)

# Instantiate the grid search model
grid_search_final2 = GridSearchCV(estimator = rf, scoring = 'balanced_accuracy', param_grid = param_grid, 
                                 cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [32]:
grid_search_final2.fit(train_features, train_labels);

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 170 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:   34.8s finished


In [33]:
grid_search_final.best_params_

{'bootstrap': False,
 'max_depth': 17,
 'max_features': 'log2',
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'n_estimators': 180}

In [34]:
grid_search_final2.best_params_

{'bootstrap': False,
 'max_depth': 15,
 'max_features': 'log2',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 150}

In [35]:
best_grid_final2 = grid_search_final2.best_estimator_
grid_final_accuracy2 = evaluate(best_grid_final2, test_features, test_labels)

Accuracy = 0.63%.
balanced_accuracy:  0.27925685425685426


In [36]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_final_accuracy2 - grid_final_accuracy) / grid_final_accuracy))

Improvement of -3.08%.
